## Разреженный автоэнкодер

До этого мы говорили о том, что построение эффективного автоэнкодера возможно только в случае, когда размер латентного слоя сильно меньше входных слоев. 

Что если мы сделаем автоэнкодер с размером латентного пространства больше входной размерности? 

При такой прямой реализации у нас ничего не получится, энкодер будет просто перемещать входные данные полностью в латентное пространство, без изменений. Наш автоэнкодер выучит, что ему нужно входные данные просто скопировать в латентное пространство, а потом перекопировать на вывод, т.е. он ничего не выучит.

Однако сделать латентный слой «узким местом» сети можно и иначе. Идея такая: мы добавим loss на латентный слой, который будет заключаться в том, что для каждого объекта использовалось ограниченное количество нейронов. 

Это позволяет модели самой выбирать размер латентного представления и использовать для разных групп объектов латентные представления разного размера, что может быть полезно для части задач и позволит получить более полезное внутреннее представление.

Такой автоэнкодер называется **разреженным** (**sparse autoencoder**).

<img src="https://edunet.kea.su/repo/EduNet-content/L14/out/sparse_autoencoder.png" alt="alttext" width = "500"/>

Для начала нам нужно задать понятие активированности нейрона. По логике это должна быть величина от 0 до 1.

0 соответствует полное отсутствие активации, 1 соответствует полная активация нейрона. Нам не подходит решение — применить сигмоиду к активациям, так как они могут быть и отрицательны, а сигмоида на отрицательных активациях стремится к нулю. А нам нужно, чтобы 0 соответствовало изначально нулевое значение. 

Потому можем сделать следующее:

 1. Берем латентный слой автоэнкодера и считаем абсолютные значения активаций. 
 2. Применяем к этим значениям сигмоиду. Теперь 0 соответствует 0.5. Полученные значения гарантированно не меньше 0.5
 3. Вычтем из этих значений 0.5 и домножим результат на 2. Теперь они распределены так, как нам нужно. 


In [ ]:
def to_01_activation(latent):
    activations = (torch.sigmoid(latent.abs()) - 0.5) * 2
    return activations

Зададим то, как для нашего автоэнкодера должен подсчитываться лосс. Он будет состоять из двух частей, первая отвечает за реконструкцию (восстановление) изображения, а вторая штрафует за включение нейронов больше некоторого порога, который мы регулируем с помощью beta

In [ ]:
def sparse_ae_loss_handler(data, recon, latent, beta=0.1, *args, **kwargs):
    activations = to_01_activation(latent)
    return (
        F.binary_cross_entropy(recon, data)
        + F.l1_loss(activations, torch.zeros_like(activations)) * beta
    )

Создадим разреженный автоэнкодер, с размером латентного пространства больше, чем размерность входных данных

In [ ]:
torch.manual_seed(42)

latent_dim = 30 * 30

learning_rate = 1e-4
encoder = Encoder(latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim)

encoder = encoder.to(device)
decoder = decoder.to(device)

optimizer = optim.Adam(
    chain(encoder.parameters(), decoder.parameters()), lr=learning_rate
)

In [ ]:
from functools import partial

for i in range(1, 6):
    train(
        enc=encoder,
        dec=decoder,
        optimizer=optimizer,
        loader=train_loader,
        epoch=i,
        single_pass_handler=ae_pass_handler,
        loss_handler=partial(
            sparse_ae_loss_handler, beta=0.01
        ),  # regulize beta parameter
        log_interval=450,
    )

In [ ]:
encoder = encoder.eval()
decoder = decoder.eval()

In [ ]:
run_res = run_eval(encoder, decoder, test_noised_loader, ae_pass_handler)
plot_digits(run_res["real"][0:9], run_res["reconstr"][0:9])

Видим, что разреженный автоэнкодер, несмотря на большой размер латентного слоя, все равно эффективно убирает шум из изображений.

Посмотрим, как активируются нейроны латентного слоя для каждого класса.

In [ ]:
run_res = run_eval(encoder, decoder, test_loader, ae_pass_handler)

In [ ]:
_, axs = plt.subplots(nrows=2, ncols=5, figsize=(16, 5))
activations = to_01_activation(torch.from_numpy(run_res["latent"])).numpy()

up_lim = activations.max()
for label in range(0, 10):
    figure = activations[run_res["labels"] == label].mean(axis=0)
    figure = figure.reshape(30, 30)
    ax = axs[label % 2, label % 5]
    ax.imshow(figure, cmap="Greys", clim=(0, 0.5))
    ax.grid(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False);

Видим, что нейронов активируются в среднем крайне мало, между классами активирующиеся нейроны отличаются.

In [ ]:
import matplotlib as mpl


def plot_percent_hist(ax, data, bins):
    counts, _ = np.histogram(data, bins=bins)
    widths = bins[1:] - bins[:-1]
    x = bins[:-1] + widths / 2
    ax.bar(x, counts / len(data), width=widths * 0.8)
    ax.xaxis.set_ticks(bins)
    ax.yaxis.set_major_formatter(
        mpl.ticker.FuncFormatter(
            lambda y, position: "{}%".format(int(np.round(100 * y)))
        )
    )
    ax.grid(True)


def plot_activations_histogram(activations, height=1, n_bins=10):
    activation_means = activations.mean(axis=0)

    mean = activation_means.mean()
    bins = np.linspace(0, 1, n_bins + 1)

    fig, [ax1, ax2] = plt.subplots(figsize=(10, 3), nrows=1, ncols=2, sharey=True)
    plot_percent_hist(ax1, activations.ravel(), bins)
    ax1.plot(
        [mean, mean], [0, height], "k--", label="Overall Mean = {:.2f}".format(mean)
    )
    ax1.legend(loc="upper center", fontsize=14)
    ax1.set_xlabel("Activation")
    ax1.set_ylabel("% Activations")
    ax1.axis([0, 1, 0, height])
    plot_percent_hist(ax2, activation_means, bins)
    ax2.plot([mean, mean], [0, height], "k--")
    ax2.set_xlabel("Neuron Mean Activation")
    ax2.set_ylabel("% Neurons")
    ax2.axis([0, 1, 0, height])

In [ ]:
plot_activations_histogram(activations, height=1.0)

Посмотрим, что у нас наблюдается в целом. 
Видно, что средний размер активации — в районе 0.10. Причем часто активация равна 0. 
Для нейронов наблюдается похожая картина: в среднем активация нейрона по всему тестовому датасету расположена в районе 0.10 и нейроны очень редко отходят от этой области. 

Вообще говоря, нам нужна именно правая картина. Левая — лишь побочный результат применения нами L1-loss.

Подход с L1-loss очень просто реализуется, но в то же время не совсем очевидно, как с помощью него задать условие вида: пусть в среднем на латентном слое активируется 1% нейронов. Понятно, что это косвенно задается величиной коэффициента $\beta$, но хотелось бы задавать это в явном виде. 

### Дивергенция Кульбака-Лейблера

Для данной цели используется дивергенция Кульбака-Лейблера, которая считается по формуле: 

$$KL(P||Q) = \int_X p(x)\log \dfrac {p(x)} {q(x)} dx$$

В теории информации $p$ считается целевым (истинным) распределением, а $q$ — тем, с которым мы его сравниваем (проверяемым). 
Важно понимать, что $KL$ не является мерой расстояния, т.к. в общем случае $KL(P||Q) != KL(Q||P)$.


[Оказывается](https://math.stackexchange.com/questions/90537/what-is-the-motivation-of-the-kullback-leibler-divergence), в подобных задачах она, как правило, обеспечивает бОльшую сходимость к требуемому распределению, нежели та же L1-регуляризация.


KL-дивергенция для $p=0.1$

<img src="https://edunet.kea.su/repo/EduNet-content/L14/out/kl_plot.png" alt="alttext" width="350">


Пусть мы хотим, чтобы на каждом слое для данного объекта активировалось в среднем p% нейронов.

В нашем случае для каждой активации i-го нейрона латентного слоя $a_i^{latent}$ мы можем решить, активирован нейрон или нет. Мы можем посчитать для каждого объекта, какая доля нейронов активировалась в его случае. 

Или же мы можем усреднить активации нейронов, если активации распределены на отрезке от 0 до 1 (например, мы можем преобразовать активации, как это сделали выше). Получим величину $\hat{p}$.

Фактически мы сравниваем два бернулиевских распределения: то, которое хотим мы, с параметром p, и то, которое мы наблюдаем — с оценочным параметром $\hat{p}$. 

$$KL(P||Q) =  p(x) \log \dfrac {p(x)} {\hat{p}(x)} + (1 - p(x)) \log \dfrac {(1 - p(x))} {1 - \hat{p}(x)} $$

Далее лишь остается просуммировать данный loss по батчу.

Можно делать и иначе — требовать, чтобы каждый нейрон в среднем активировался в p% случаев. В этом случае на первом шаге мы усредняем активации не по всему слою, а по батчам. А вот подсчитанный loss усредняем по всем нейронам слоя.

Чем KL-loss лучше L1-loss и L2-loss? 
Он позволяет нам приближаться к решению более плавно и четко регулировать долю активирующихся нейронов.

In [ ]:
plt.figure(figsize=(7, 7))
p = 0.1
q = np.linspace(0.001, 0.999, 500)
kl_div = p * np.log(p / q) + (1 - p) * np.log((1 - p) / (1 - q))
mse = (p - q) ** 2
mae = np.abs(p - q)
plt.plot([p, p], [0, 0.3], "k:")
plt.text(0.05, 0.32, "Target\nsparsity", fontsize=14)
plt.plot(q, kl_div, "b-", label="KL divergence")
plt.plot(q, mae, "g--", label=r"MAE ($\ell_1$)")
plt.plot(q, mse, "r--", linewidth=1, label=r"MSE ($\ell_2$)")
plt.legend(loc="upper left", fontsize=14)
plt.xlabel("Actual sparsity")
plt.ylabel("Cost", rotation=0)
plt.axis([0, 1, 0, 0.95]);